In [1]:
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt 
import glob
import os
import xarray as xr 
import pandas as pd
import numpy as np


from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=12,                     # Number of cores per task (adjust if needed)
    processes=6,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)


from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:8787/status,
Dashboard: http://10.42.239.61:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:34237,Workers: 0
Dashboard: http://10.42.239.61:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
met_arm = '/home1/nalex2023/Datasets/interpsonde_manus'


def create_dft(in_dir,arm_data=True):
    files = glob.glob(in_dir+os.sep+'*.nc')
    if len(files) == 0:
        files = glob.glob(in_dir+os.sep+'*.cdf')
        if len(files) == 0:
            raise Exception('No files Found')
        else:
            pass
    else:
        pass
    
    dft = pd.DataFrame(files,columns=['filepath'])
    dft['filename'] = dft['filepath'].str.split(os.sep).str[-1]
    
    if arm_data:
        dft['datetime'] = pd.to_datetime(dft['filename'].str.split(
            '.',
            expand=True).iloc[:, 2] + dft['filename'].str.split(
                '.',  expand=True).iloc[:, 3],format='%Y%m%d%H%M%S')


    return dft


dft_met = create_dft(met_arm,arm_data=False)

years = np.arange(2000,2013)


dset = xr.open_mfdataset(dft_met['filepath'].values,combine='by_coords')

In [14]:
dset_diurnal_complete = dset.groupby('time.season').apply(lambda x: x.groupby('time.hour').mean())

dset_diurnal_complete

<xarray.Dataset> Size: 766kB
Dimensions:   (season: 4, hour: 24, height: 332)
Coordinates:
  * height    (height) float32 1kB 0.004 0.024 0.044 0.064 ... 39.5 40.0 40.5
  * hour      (hour) int64 192B 0 1 2 3 4 5 6 7 8 ... 15 16 17 18 19 20 21 22 23
  * season    (season) object 32B 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    bar_pres  (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
    temp      (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
    dp        (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
    rh        (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
    u_wind    (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
    v_wind    (season, hour, height) float32 127kB dask.array<chunksize=(1, 1, 332), meta=np.ndarray>
Attributes: (12/13)
    command_line:          idl -D 0 -R -n interpolatedsonde -s twp -f C1 -b 2...
    Conventions:           ARM-1.1
    process_version:       vap-interpolatedsonde-6.6-0.el6
    input_datastreams:     twpgriddedsondeC1.c0 : 2.2 : 20010403.000030-20010...
    dod_version:           interpolatedsonde-c1-4.0
    site_id:               twp
    ...                    ...
    facility_id:           C1
    data_level:            c1
    location_description:  Tropical Western Pacific (TWP), Manus Island, Papu...
    datastream:            twpinterpolatedsondeC1.c1
    doi:                   10.5439/1095316
    history:               created by user ttoto on machine chalk at 2016-05-...

In [15]:
out_dir = '/home1/nalex2023/Datasets/LSB_OUTS/'

dset_diurnal_complete.to_netcdf(out_dir+'diurnal_climate_interp_seasonal_2001_2014.nc')

In [16]:
client.close()
cluster.close()